In [1]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation
from keras_tuner.tuners import RandomSearch

In [2]:
#load fashin mnist dataset
(training_images, training_labels), (test_images, test_labels) = fashion_mnist.load_data()
print(training_images.shape)

#reshape
training_images = training_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)
print(training_images.shape)

(60000, 28, 28)
(60000, 28, 28, 1)


In [3]:

def build_model_cnn(hp):  # random search passes this hyperparameter() object 
    model_cnn = keras.models.Sequential()
    
    model_cnn.add(Conv2D(hp.Int('input_units',
                                min_value=32,
                                max_value=256,
                                step=32), (3, 3), input_shape=training_images.shape[1:]))
    
    model_cnn.add(Activation('relu'))
    model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

    for i in range(hp.Int('n_layers', 1, 4)):  # adding variation of layers.
        model_cnn.add(Conv2D(hp.Int(f'conv_{i}_units',
                                min_value=32,
                                max_value=256,
                                step=32), (3, 3)))
        model_cnn.add(Activation('relu'))

    model_cnn.add(Flatten()) 
    model_cnn.add(Dense(128))
    model_cnn.add(Dense(10))
    model_cnn.add(Activation("softmax"))

    model_cnn.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model_cnn


In [4]:
import time
LOG_DIR = f"{int(time.time())}"

tuner = RandomSearch(
    build_model_cnn,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=1, 
    directory=LOG_DIR)

tuner.search(x=training_images,
             y=training_labels,
             verbose=2,
             epochs=1,
             batch_size=64,
             validation_data=(test_images, test_labels))

Trial 1 Complete [00h 00m 16s]
val_accuracy: 0.8481000065803528

Best val_accuracy So Far: 0.8481000065803528
Total elapsed time: 00h 00m 16s
INFO:tensorflow:Oracle triggered exit


In [5]:
tuner.get_best_hyperparameters()[0].values

{'conv_0_units': 128,
 'conv_1_units': 32,
 'conv_2_units': 32,
 'input_units': 160,
 'n_layers': 3}

In [6]:
tuner.get_best_models()[0].summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 160)       1600      
                                                                 
 activation (Activation)     (None, 26, 26, 160)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 160)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 128)       184448    
                                                                 
 activation_1 (Activation)   (None, 11, 11, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 32)          36896     
                                                        